<a href="https://colab.research.google.com/github/apragupta/ML1_Final_Proj/blob/main/FP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drive setup

In [ ]:
import os
from google.colab import drive, auth
import pandas as pd
import math
import seaborn as sns
# Mount Google Drive - This will prompt you to click a link to authorize
drive.mount('/content/drive')

# Navigate to directory and list files
os.chdir('drive/My Drive/Year 2 NEU/Spring 2021/Machine Learning 1/Final Project')
os.listdir('.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['FP_2.ipynb',
 'images_filtered.zip',
 'FP.ipynb',
 'cleaned_filtered.csv',
 'checkpoints',
 'Image_Labels_Subset_Train_GCC-Labels-training (1).tsv',
 'images_filtered',
 'senticap_dataset.json',
 'senticap.json',
 'val2014',
 'senti_cap_sent.csv',
 'FP_1.ipynb',
 'sent_added',
 'feature_set',
 'FP_3.ipynb']

# TF setup

In [ ]:
import tensorflow as tf

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import collections
import random
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
from tqdm import tqdm

In [ ]:
os.listdir()

['FP_2.ipynb',
 'images_filtered.zip',
 'FP.ipynb',
 'cleaned_filtered.csv',
 'checkpoints',
 'Image_Labels_Subset_Train_GCC-Labels-training (1).tsv',
 'images_filtered',
 'senticap_dataset.json',
 'senticap.json',
 'val2014',
 'senti_cap_sent.csv',
 'FP_1.ipynb',
 'sent_added',
 'feature_set',
 'FP_3.ipynb']

In [ ]:
# os.listdir('images_filtered')

In [ ]:
sentiment = pd.read_csv('sent_added', index_col=0)

In [ ]:
sentiment

,caption,link,objects,mid,object_confidence,pos,image_path,size,hf_sent,hf_sent_class,hf_sent_conf
index,,,,,,,,,,,
0,snowman on a red background,https://thumb10.shutterstock.com/display_pic_w...,"snowman,cartoon,christmas eve,greeting card,sa...","/m/0152hh,/m/0215n,/m/014r1s,/m/03r8dh,/m/027g...","0.9607862830162048,0.9256524443626404,0.869515...",0,data\initial_sample\images\0.png,"(450, 470)","[{'label': 'POSITIVE', 'score': 0.983012139797...",POSITIVE,0.983012
1,celebrity celebrates with his team mates after...,https://media.gettyimages.com/photos/david-bec...,"player,sports,tournament,sports equipment,team...","/m/02vzx9,/m/06ntj,/m/012sbd,/m/05y5lj,/m/03x0...","0.995377242565155,0.9889867305755615,0.9811984...",1,data\initial_sample\images\1.png,"(477, 612)","[{'label': 'POSITIVE', 'score': 0.999717175960...",POSITIVE,0.999717
2,"action shot of basketball player , passing the...",https://onwisconsin.uwalumni.com/content/uploa...,"sports,team sport,ball game,player,sport venue...","/m/06ntj,/m/03x045,/m/020m7t,/m/02vzx9,/m/0bmg...","0.9889867305755615,0.9755526781082153,0.974188...",2,data\initial_sample\images\2.png,"(1200, 801)","[{'label': 'POSITIVE', 'score': 0.998242616653...",POSITIVE,0.998243
3,a sunny winter day by lake .,https://s-media-cache-ak0.pinimg.com/originals...,"sky,snow,winter,natural landscape,nature,blue,...","/m/01bqvp,/m/06_dn,/m/086mh,/m/03d28y3,/m/05h0...","0.9884884357452393,0.9849196076393127,0.983304...",3,data\initial_sample\images\3.png,"(640, 640)","[{'label': 'POSITIVE', 'score': 0.999814331531...",POSITIVE,0.999814
4,"the extraordinary domed ceiling , with an arti...",https://i.pinimg.com/736x/7d/7a/bd/7d7abd23c2d...,"ceiling,chandelier,light fixture,lighting,arch...","/m/03gfsp,/m/02ws6z,/m/08jpwt,/m/019sc6,/m/03n...","0.9757712483406067,0.9078124165534973,0.849791...",4,data\initial_sample\images\4.png,"(467, 700)","[{'label': 'POSITIVE', 'score': 0.999864161014...",POSITIVE,0.999864
...,...,...,...,...,...,...,...,...,...,...,...
9625,adapted work to the universe .,https://i.pinimg.com/736x/b6/85/d9/b685d9287f3...,"text,sky,font,astronomical object,universe,atm...","/m/07s6nbt,/m/01bqvp,/m/03gq5hm,/m/01d9ll,/m/0...","0.9626815915107727,0.9585431814193726,0.945139...",9625,data\initial_sample\images\9625.png,"(500, 494)","[{'label': 'POSITIVE', 'score': 0.998943090438...",POSITIVE,0.998943
9626,praying and folding hands over poetry book,https://media.istockphoto.com/photos/praying-a...,"white,black,hand,black-and-white,finger,monoch...","/m/083jv,/m/019sc,/m/0k65p,/m/01g6gs,/m/09cx8,...","0.9628557562828064,0.9571874141693115,0.937342...",9626,data\initial_sample\images\9626.png,"(612, 408)","[{'label': 'NEGATIVE', 'score': 0.979139387607...",NEGATIVE,0.979139
9627,cute hand drawn seamless pattern with circles ...,https://thumb7.shutterstock.com/display_pic_wi...,"text,circle,font,pattern,line,design,number,il...","/m/07s6nbt,/m/01vkl,/m/03gq5hm,/m/0hwky,/m/03s...","0.9523645639419556,0.9162501692771912,0.908029...",9627,data\initial_sample\images\9627.png,"(450, 470)","[{'label': 'POSITIVE', 'score': 0.999741852283...",POSITIVE,0.999742


# Preprocessing with MobileNet


In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [ ]:
image_model = tf.keras.applications.MobileNet()

new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [ ]:
from tqdm import tqdm

In [ ]:
def pos_to_imgP(pos):
  return f'images_filtered/{pos}.png'

img_name_vector = []

for pos in list(sentiment.index.values):
  path = pos_to_imgP(pos)
  img_name_vector.append(path)

# Get unique images
encode_train = sorted(set(img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(16)

for img, path in tqdm(image_dataset):

  batch_features = image_features_extract_model(img)

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8") + str('.mobile')
    np.save(path_of_feature, bf.numpy())

# Soemtibg

In [ ]:
test = np.load('images_filtered/9626.png.npy')

In [ ]:
test.shape

(64, 2048)

In [ ]:
test1 = np.load('images_filtered/0.png.mobile.npy')

In [ ]:
test1.shape

(1000,)

In [ ]:
feature_set = pd.DataFrame(columns=[i for i in range(1000)])
feature_set['sentiment'] = []

for i, row in sentiment.iterrows():
  print(i)
  index = row.name
  cur = np.load(f'images_filtered/{index}.png.mobile.npy')
  to_add = cur.flatten().tolist()
  to_add.append(row['hf_sent_class'])
  feature_set.loc[i] = to_add


Streaming output truncated to the last 5000 lines.
4451
4452
4453
4454
4455
4456
4457
4458
4460
4461
4462
4463
4464
4465
4466
4467
4468
4469
4470
4471
4472
4473
4474
4475
4476
4477
4478
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4493
4495
4496
4497
4498
4499
4500
4501
4502
4503
4504
4505
4506
4507
4508
4509
4510
4511
4512
4513
4514
4515
4516
4517
4518
4519
4520
4521
4522
4523
4524
4525
4526
4528
4529
4530
4531
4532
4533
4534
4535
4536
4537
4538
4539
4540
4541
4543
4544
4545
4546
4547
4548
4549
4550
4551
4552
4553
4554
4555
4556
4557
4558
4559
4560
4561
4562
4563
4564
4565
4566
4567
4568
4569
4570
4571
4572
4573
4574
4575
4576
4577
4578
4579
4580
4581
4582
4583
4585
4586
4587
4588
4589
4590
4591
4592
4593
4594
4595
4596
4597
4599
4600
4601
4602
4603
4604
4605
4606
4607
4608
4609
4610
4611
4612
4613
4614
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4627
4628
4629
4631
4633
4634
4635
4636
4637
4638
4639
4640
4641
4642
4643
4644
4645
4646
4647
4648
4649

In [ ]:
feature_set

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,sentiment
0,1.825437e-07,3.081069e-06,6.151669e-07,6.222912e-07,3.725395e-07,3.865697e-05,6.937968e-08,1.620700e-05,3.596125e-06,1.337872e-07,8.843734e-05,4.655684e-06,1.832618e-06,5.039533e-06,9.255446e-06,2.246210e-06,7.294258e-04,9.038307e-05,4.110755e-06,9.945628e-05,6.501691e-05,1.951231e-05,9.060253e-07,2.130383e-06,1.092555e-06,4.047858e-06,6.023028e-05,3.098373e-05,6.033677e-06,5.805536e-06,1.708504e-06,2.678744e-06,2.970520e-05,1.844297e-07,1.559572e-06,4.176057e-07,1.100943e-06,1.938812e-07,6.542270e-05,7.224590e-08,...,2.901232e-05,1.940366e-05,1.165480e-07,3.043390e-05,2.661321e-06,2.040870e-04,6.865062e-04,6.120695e-05,5.304710e-03,1.058960e-04,1.000266e-05,4.066160e-05,2.502704e-04,5.857625e-06,5.890674e-05,1.585992e-07,9.963989e-07,3.995418e-04,6.530394e-07,5.081920e-05,8.371006e-04,5.926265e-06,4.647442e-06,5.058517e-05,5.015725e-05,8.258094e-04,1.392520e-05,7.575899e-06,3.018130e-06,1.254096e-06,5.359753e-06,3.610757e-06,9.109829e-07,3.424403e-05,2.785006e-04,3.497414e-04,4.637303e-06,3.418282e-05,0.000147,POSITIVE
1,9.743706e-10,3.255081e-07,7.856821e-08,2.840356e-09,3.959591e-07,4.443392e-09,2.565245e-09,5.025527e-08,3.903430e-09,2.469008e-07,4.924568e-07,4.365052e-07,8.465611e-09,7.647478e-08,6.199613e-09,7.457772e-07,1.585488e-07,1.849024e-08,1.053838e-07,8.831719e-09,3.642966e-09,2.966703e-09,1.952014e-07,1.680321e-07,2.713867e-08,9.944913e-09,6.538125e-09,1.491354e-09,9.956842e-07,2.976480e-09,1.488291e-08,1.188868e-08,2.683461e-09,6.199190e-08,3.418475e-08,1.018170e-07,1.676259e-07,1.886993e-08,2.754686e-08,1.156055e-08,...,1.680018e-06,2.198997e-09,4.194494e-08,2.478991e-09,7.773210e-09,6.077891e-06,8.932924e-06,7.445165e-07,7.559190e-08,4.021039e-08,6.701115e-06,4.426876e-07,3.854181e-07,1.219906e-07,1.240589e-07,4.509332e-08,2.188769e-06,1.066551e-06,4.517138e-09,1.519828e-06,6.997589e-01,2.334237e-05,1.053775e-05,7.682678e-09,1.164126e-07,1.551132e-09,9.094554e-08,1.917048e-09,2.982752e-07,4.593377e-09,2.317335e-08,7.678064e-09,2.791685e-08,4.743430e-08,1.611797e-08,6.253970e-10,1.991041e-08,1.679559e-08,0.000002,POSITIVE
2,9.597818e-09,5.120547e-06,1.261530e-06,6.598006e-07,1.151605e-07,5.274585e-07,1.298079e-07,4.675381e-09,2.819261e-08,8.169543e-08,1.083971e-06,3.002534e-06,3.093216e-07,1.442714e-07,1.366039e-07,4.623658e-07,1.402487e-06,3.970051e-08,1.132931e-08,3.031383e-08,1.816143e-07,2.663476e-08,8.411089e-07,3.151740e-06,2.916771e-08,1.629456e-07,2.073944e-07,3.961776e-08,1.356191e-07,7.080595e-08,8.565229e-07,4.058273e-06,1.896106e-07,7.613246e-06,3.898921e-07,4.763010e-07,3.934424e-07,3.019887e-08,6.950574e-08,1.654743e-08,...,3.827929e-08,2.186444e-08,6.628717e-08,9.925559e-08,2.112798e-08,1.428263e-07,8.316859e-08,3.577851e-07,3.503696e-07,6.865386e-06,4.277629e-05,4.910936e-06,1.148648e-05,3.399977e-08,8.006339e-05,4.686343e-06,2.362347e-05,6.000756e-05,1.490726e-05,5.801792e-06,1.305673e-02,2.948562e-06,9.668110e-06,2.451997e-06,3.147012e-06,8.116123e-06,4.312099e-07,8.462425e-08,9.509935e-08,3.647951e-07,3.106075e-07,9.817934e-08,3.188724e-08,3.403931e-07,1.808005e-07,1.989944e-07,4.030477e-07,1.957145e-07,0.000001,POSITIVE
3,3.576900e-06,1.137077e-06,3.392912e-06,4.956223e-07,3.598596e-06,2.840574e-05,8.567486e-06,1.668360e-06,1.695566e-06,1.311694e-05,5.366268e-07,2.751898e-07,4.683070e-07,2.717718e-06,9.243219e-07,1.011731e-06,3.075690e-07,7.056495e-06,4.372628e-06,8.791137e-07,2.794894e-06,2.506775e-06,2.026819e-05,1.226766e-05,3.039562e-05,2.121414e-07,1.506391e-07,1.143929e-06,3.671373e-07,5.045163e-07,7.446660e-07,1.117263e-07,2.205233e-07,1.883190e-05,2.240813e-05,1.784171e-06,4.505926e-06,2.137570e-06,1.327328e-07,4.961411e-07,...,1.099388e-06,9.186014e-08,2.559548e-07,6.368066e-08,1.925686e-06,1.072664e-04,1

In [ ]:
feature_set.to_csv('feature_set')

In [ ]:
os.listdir()

['FP_2.ipynb',
 'images_filtered.zip',
 'FP.ipynb',
 'cleaned_filtered.csv',
 'checkpoints',
 'Image_Labels_Subset_Train_GCC-Labels-training (1).tsv',
 'images_filtered',
 'senticap_dataset.json',
 'senticap.json',
 'val2014',
 'senti_cap_sent.csv',
 'FP_1.ipynb',
 'sent_added',
 'feature_set',
 'FP_3.ipynb']